In [1]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
# Untuk pengolahan data
import pandas as pd
import numpy as np
from zipfile import ZipFile
from pathlib import Path

# Untuk visualisasi data
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
sns.set_palette('Set1')
sns.set()

# Untuk pemodelan
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.metrics import RootMeanSquaredError
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from haversine import haversine, Unit
from PIL import Image

# Untuk menghilangkan warnings saat plotting seaborn
import warnings
warnings.filterwarnings('ignore')

# Untuk mengupload file
import os

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")
place = pd.read_excel(r'/content/gdrive/My Drive/Colab Notebooks/Dataset_Restaurant.xlsx')

Mounted at /content/gdrive


In [4]:
data=place.copy()

In [5]:
data

,Place_ID,Place,Image,URL,Rating,Num_Rating,Category,Address,Longitude,Latitude,Label_Kecamatan
0,11,Mr.K Cafe Gombel,https://lh5.googleusercontent.com/p/AF1QipOh43...,https://www.google.com/maps/place/Mr.K+Cafe+Go...,4.5,6235,Restoran,Jl. Setia Budi No.28,110.418685,-7.038795,1
1,12,Super Penyet Setiabudi,https://lh5.googleusercontent.com/p/AF1QipMufn...,https://www.google.com/maps/place/Super+Penyet...,4.4,5702,Ayam,Jl. Setia Budi No.207,110.411722,-7.058634,1
2,13,Sim Six (Sim-6) garden Resto Ngesrep timur,https://lh5.googleusercontent.com/p/AF1QipPfQx...,https://www.google.com/maps/place/Sim+Six+%28S...,4.5,4265,Restoran,Jl. Ngesrep Tim. V No.25,110.417238,-7.064682,1
3,14,Sowak (Sop Iwak),https://lh5.googleusercontent.com/p/AF1QipPKYy...,https://www.google.com/maps/place/Sowak+%28Sop...,4.4,4251,Rumah Makan,Jl. Banjarsari Selatan No.18,110.432990,-7.061215,1
4,15,Warung Lesehan Fifty Fifty Sumurboto Banyumanik,https://lh5.googleusercontent.com/p/AF1QipNxvJ...,https://www.google.com/maps/place/Warung+Leseh...,4.3,3822,Jawa,Jl. Ngesrep Tim. V No.118,110.426369,-7.053008,1
...,...,...,...,...,...,...,...,...,...,...,...
154,166,Ayam Penyet Surabaya,https://lh5.googleusercontent.com/p/AF1QipOKkb...,https://www.google.com/maps/place/Ayam+Penyet+...,4.4,1118,Ayam Penyet,Jl.Walisongo Km 9 No.99A,110.342095,-7.781593,16
155,167,Kuliner Lor Tugu,https://lh5.googleusercontent.com/p/AF1QipMv-b...,https://www.google.com/maps/place/Kuliner+Lor+...,4.6,1065,Restoran,Jl. A.M. Sangaji,110.365050,-7.782046,16
156,168,RM Iwak Manuk Hj. Jummi,https://lh5.googleusercontent.com/p/AF1QipPjU4...,https://www.google.com/maps/place/RM+Iwak+Manu...,4.4,732,Rumah Makan,Jl. Semarang - Kendal KM. 12,110.319835,-6.975715,16
157,169,Antarakata Tugu Muda,https://lh5.googleusercontent.com/p/AF1QipMDEc...,https://www.google.com/maps/place/Antarakata+T...,4.6,660,Restoran,"2C85+596, Jl. HOS Cokroaminoto",110.405800,-6.984577,16


In [36]:
places = data['Place'].values
longitude = data['Longitude'].values
latitude = data['Latitude'].values
addresses = data['Address'].values
ratings = data['Rating'].values
num_ratings = data['Num_Rating'].values
category = data['Category'].values
image_paths = data['Image'].values
url_paths = data['URL'].values

In [7]:
label_encoder = LabelEncoder()
encoded_places = label_encoder.fit_transform(places)

In [13]:
input_data = np.column_stack((latitude, longitude))

In [14]:
model = keras.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=[2]),  # Longitude and Latitude features
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(len(label_encoder.classes_), activation='softmax')  # Output probabilities for each place
])

In [15]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Train the model
epochs = 500
history = model.fit(input_data, encoded_places, epochs=epochs, verbose=1)

Epoch 1/500
5/5 [==============================] - 1s 3ms/step - loss: 5.2685
Epoch 2/500
5/5 [==============================] - 0s 3ms/step - loss: 5.0929
Epoch 3/500
5/5 [==============================] - 0s 4ms/step - loss: 5.0623
Epoch 4/500
5/5 [==============================] - 0s 3ms/step - loss: 5.0316
Epoch 5/500
5/5 [==============================] - 0s 3ms/step - loss: 5.0184
Epoch 6/500
5/5 [==============================] - 0s 3ms/step - loss: 5.0109
Epoch 7/500
5/5 [==============================] - 0s 3ms/step - loss: 5.0079
Epoch 8/500
5/5 [==============================] - 0s 3ms/step - loss: 5.0065
Epoch 9/500
5/5 [==============================] - 0s 4ms/step - loss: 5.0044
Epoch 10/500
5/5 [==============================] - 0s 4ms/step - loss: 5.0040
Epoch 11/500
5/5 [==============================] - 0s 3ms/step - loss: 5.0035
Epoch 12/500
5/5 [==============================] - 0s 4ms/step - loss: 5.0031
Epoch 13/500
5/5 [==============================] - 0s 3ms/st

In [33]:
# User location
user_location = [-6.975050, 110.42657]  # Example user location

# Calculate distances between user location and all places
user_distances = [round(haversine((user_location[0], user_location[1]), (lat, lon), unit=Unit.KILOMETERS), 2) for lat, lon in input_data]

In [39]:
recommendations = []
for i, distance in enumerate(user_distances):
    if places[i] != 'Current Location':  # Exclude places that are the same as user's current location
        recommendations.append({
            'Place': places[i],
            'Address': addresses[i],
            'Distance': distance,
            'Rating': ratings[i],
            'Num_Ratings': num_ratings[i],
            'Category': category[i],
            'Image_Path': image_paths[i],
            'Url_Path': url_paths[i]
        })

recommendations.sort(key=lambda x: x['Distance'])  # Sort recommendations by distance

print("Top Recommendations:")
for recommendation in recommendations[:10]:  # Show top 10 recommendations
    print("Place:", recommendation['Place'])
    print("Address:", recommendation['Address'])
    print("Distance:", recommendation['Distance'], "Km")
    print("Rating:", recommendation['Rating'])
    print("Number of Ratings:", recommendation['Num_Ratings'])
    print("Category:", recommendation['Category'])
    print("Image Path:", recommendation['Image_Path'])
    print("Url Path:", recommendation['Url_Path'])
    print()

Top Recommendations:
Place: Pakne Heksa Sidorejo
Address: Jl. Sidorejo No.2-82
Distance: 0.41 Km
Rating: 4.6
Number of Ratings: 1719
Category: Rumah Makan
Image Path: https://lh5.googleusercontent.com/p/AF1QipMbxFu-nLvvQoGlo19CpQkRL_OzCxDPlUtDdtjo=w122-h92-k-no
Url Path: https://www.google.com/maps/place/Pakne+Heksa+Sidorejo/data=!4m7!3m6!1s0x2e708cab9b33e82b:0x17db799d80d25853!8m2!3d-6.9779502!4d110.4314025!16s%2Fg%2F11d_1lywbm!19sChIJK-gzm6uMcC4RU1jSgJ152xc?authuser=0&hl=id&rclk=1

Place: Nasi Gandul Pak Memet
Address: Jalan Dokter Cipto No.12
Distance: 0.73 Km
Rating: 4.6
Number of Ratings: 5080
Category: Jawa Tengah
Image Path: https://lh5.googleusercontent.com/p/AF1QipNvrJGiCMHFW73paeK96I8ysoz0fwj1yf3cDmM=w80-h96-k-no
Url Path: https://www.google.com/maps/place/Nasi+Gandul+Pak+Memet/data=!4m7!3m6!1s0x2e708ca38456ffe1:0x1b2475452707a84a!8m2!3d-6.9707446!4d110.4341259!16s%2Fg%2F1pzsrwcjl!19sChIJ4f9WhKOMcC4RSqgHJ0V1JBs?authuser=0&hl=id&rclk=1

Place: Nasi Gandul Pak Memet
Address: Ja